In [1]:
import sys
import os
import numpy as np
import pandas as pd
from math import floor
import torch
from torch.utils.data import DataLoader

# Append the project dir to path
sys.path.append(os.path.join("..", ".."))
from data_pipeline.utils import train_test_split, create_metadata_df, get_sample_weights_of_dataset, measurements_to_df
from data_pipeline.dataset_xy import CARLADatasetXY
from data_pipeline.data_sampler import BranchPerCommandSampler

/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_data = os.path.join("..", "..", "data", "data")

config_xy = {"used_inputs": ["rgb", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

df_meta_data = create_metadata_df(path_data, config_xy["used_inputs"])

# Train/test split
train_test_config = {
    "train": ['Town00', 'Town01', 'Town02', 'Town03', 'Town04', 'Town05', 'Town07', 'Town08', 'Town09', 'Town10'],
    "test": ['Town06']
}
df_meta_data_train, df_meta_data_test_1, df_meta_data_test_2 = train_test_split(df_meta_data, towns_intersect=train_test_config) # , df_meta_data_noisy=df_meta_data_noisy
df_meta_data_train = df_meta_data_train.sample(64*50).reset_index(drop=True)

# Create Dataset & DataLoader
dataset_train = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data_train, config=config_xy)

sample_weights = get_sample_weights_of_dataset(dataset_train, num_bins=10, multilabel_option=False)

Varying number files among input types: ../../data/data/rl_dataset_23_11/Routes_clipped_Town05_rl_Seed0/clipped_Town05_rl_route30_11_23_17_42_15/lidar_trans


100%|██████████| 3200/3200 [00:01<00:00, 2022.55it/s]


In [3]:
df_meas_train = measurements_to_df(df_meta_data_train)

100%|██████████| 3200/3200 [00:00<00:00, 10203.04it/s]


In [6]:
batch_size = 6 * 9 # num commands * num samples per command per batch

In [7]:
weighted_sampler_cmd = BranchPerCommandSampler(dataset_train, df_meas_train, batch_size)

In [8]:
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, num_workers=0, shuffle=False, sampler=weighted_sampler_cmd)

In [9]:
for X, Y, IDX in dataloader_train:
    pass

In [10]:
X["command"].shape

torch.Size([54, 1])

In [11]:
X["command"].numpy().flatten()

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5., 6., 6., 6., 6., 6., 6.,
       6., 6., 6.])